In [53]:
import logging
logging.root.setLevel(logging.INFO)


In [54]:
import random

from xcs import XCSAlgorithm
from xcs.scenarios import Scenario

In [55]:
import random

from xcs.scenarios import Scenario
from xcs.bitstrings import BitString

class HaystackProblem(Scenario):
    
    def __init__(self, training_cycles=1000, input_size=500):
        self.input_size = input_size
        self.possible_actions = (True, False)
        self.initial_training_cycles = training_cycles
        self.remaining_cycles = training_cycles
        self.needle_index = random.randrange(input_size)
        self.needle_value = None

    def reset(self):
        self.remaining_cycles = self.initial_training_cycles
        haystack = BitString.random(self.input_size)
        self.needle_value = haystack[self.needle_index]
        return haystack
        
    # XCS Hosford42 functions
    @property
    def is_dynamic(self):
        return False
        
    def get_possible_actions(self):
        return self.possible_actions
        
    def more(self):
        return self.remaining_cycles > 0
    
    def sense(self):
        haystack = BitString.random(self.input_size)
        self.needle_value = haystack[self.needle_index]
        return haystack
    
    def execute(self, action):
        self.remaining_cycles -= 1
        return action == self.needle_value

    # XCS Pyalcs functions
    def step(self, action):
        done = not self.execute(action)
        rawstate = str(self.sense())
        if not done:
            reward = 0
        else:
            reward = 1
        return rawstate, reward, done, _

In [56]:
logging.root.setLevel(logging.INFO)
scenario = HaystackProblem()

In [57]:
algorithm = XCSAlgorithm()
algorithm.exploration_probability = .1
algorithm.discount_factor = 0
algorithm.do_ga_subsumption = True
algorithm.do_action_set_subsumption = True

In [58]:
model = algorithm.new_model(scenario)
model.run(scenario, learn=True)


In [59]:
print(model)
print(len(model))

00101100001#1##00#11001101##100#0#0001##0##0111101#1##0#001#00##00110###10#0#00#01##010##11#101#10#0010#0000#00#0##100010##1#0001010#1110#10001###000#10110##0##0##0#1#0110#1##1##00111##0110#111101##1#1#11011###1#001#0#001#1#010#001100###1#001#00#00##1#11100#001##1#1#1#0#1##001##00#0##0##010#111101#1#0011####11110#01#000#1##11001##0##0##011100##0101##00010100101##10#####000101###1####0001##011#001#00##01010001111#0###0010#11#1#100##1#00#0#11#01111#0#00011#0#11#1011#1##101101#100101010##001##000## => False
    Time Stamp: 934
    Average Reward: 0.0
    Error: 0.0
    Fitness: 8.5e-07
    Experience: 0
    Action Set Size: 1
    Numerosity: 1
0010#10#001#1##00#11001101##100###0001##0#10111101#1##0#001#00##00110##110#0#00#01#0010##11#101#10#0010#0000#00#0##100010#####001010#1110#10001#1#000#1011###0##0##0#1#0#10#1##11#00111##0110011#1#1##1#1#11011###1##01#0#001#1101##001100###1#001#00#00##1#111000001##1#101#001##001##00#0##0##010#111101#1##011###111110#01#00001##11001##0##0##01110###0101#

In [60]:
for rule in model:
    if rule.fitness > .5 and rule.experience >= 10:
        print(rule.condition, '=>', rule.action, ' [%.5f]' % rule.fitness)

In [61]:
from lcs.agents.xcs import XCS, Configuration

cfg = Configuration(number_of_actions=4,
                    gamma=0,
                    metrics_trial_frequency=5,
                    covering_wildcard_chance=0.99
                    )


In [62]:
agent = XCS(cfg)
explore_population, explore_metrics = agent.explore(scenario, 1000, False)

AssertionError: 

In [ ]:
print(len(explore_population))
for rule in explore_population:
    if rule.fitness > .5 and rule.experience >= 10:
        print(rule.condition, '=>', rule.action, ' [%.5f]' % rule.fitness)